# Load UCSC Cell Browser project in Vitessce

This notebook shows you how to use the `convert_cell_browser_project_to_anndata` function, which allows you to take an existing project, published in https://cells.ucsc.edu/ and:
1. Convert it into the AnnData format that is supported by Vitessce
2. Save the AnnData object as a Zarr store
3. Configure Vitessce with the AnnData-Zarr store
4. Render a Vitessce widget based on the config (step 3) directly in the notebook.

The dataset that you choose to convert needs to be a valid UCSC Cell Browser "project", accessible from https://cells.ucsc.edu/, with a configuration available in https://github.com/ucscGenomeBrowser/cellbrowser-confs

The `convert_cell_browser_project_to_anndata` function takes the name of that project as an input. For example, to convert this project, https://cells.ucsc.edu/?ds=adultPancreas, you will neeed to pass `"adultPancreas"` as the project name.

In [5]:
import os
import json
from os.path import join
from vitessce import (
    AnnDataWrapper,
    VitessceConfig,
)
from vitessce.data_utils.anndata import VAR_CHUNK_SIZE
from vitessce.data_utils.ucsc_cellbrowser import convert_cell_browser_project_to_anndata

## 1. Convert UCSC Cell Browser project to a format that is supported by Vitessce
#### Output:
An AnnData object


In [ ]:
## 3. Convert UCSC Cell Browser project to a Vitessce view config

In [22]:
# Example run, coverting "adultPancreas" project:
adata = convert_cell_browser_project_to_anndata(project_name="adultPancreas", keep_only_marker_genes=True)

Converting CellBrowser config for project adultPancreas to Anndata-Zarr object
Successfully fetched configuration: https://cells.ucsc.edu/adultPancreas/dataset.json.
CellBrowser config is valid. Proceeding further with conversion.
Successfully downloaded expression matrix https://cells.ucsc.edu/adultPancreas/exprMatrix.tsv.gz.
Loading expression matrix into Anndata object ...
This dataset uses the format identifier|symbol for the ad.obs gene names (e.g. “ENSG0123123.3|HOX3”). We are keeping only the symbol.
Adding cell metadata to Anndata object ...
Successfully downloaded metadata meta.tsv.
Successful extraction of the following coordinates and URLS: {'X_tsne': 'tMinusSNE.coords.tsv.gz'}
Adding X_tsne to Anndata object ...
X_tsne successfully added.
Done adding coordinates to the Anndata object.
Filtering out all non-marker genes from Anndata object ...
Successfully filtered out all non-marker genes from Anndata object.
About to write the Anndata object to the Zarr store. The followin

## 2. Save the AnnData object as a Zarr store

In [24]:
zarr_filepath = join("data", "out.adata.zarr")
os.makedirs(os.path.dirname(zarr_filepath), exist_ok=True)
adata.write_zarr(zarr_filepath, chunks=[adata.shape[0], VAR_CHUNK_SIZE])

## 3. Configure Vitessce with the AnnData-Zarr store

In [25]:
anndata_wrapper_inst = AnnDataWrapper(
    adata_path=zarr_filepath,
    obs_feature_matrix_path="X",
    obs_embedding_paths=["obsm/X_tsne"],
    obs_embedding_names=["t-SNE"],
    obs_set_paths=["obs/cluster", "obs/age"],
    obs_set_names=["cluster", "age"],
)
vc = VitessceConfig(schema_version="1.0.15", name="Vitessce configuration for CellBrowser project adultPancreas")
anndata_wrapper_inst.auto_view_config(vc)

## 4. Render the Vitessce widget

In [26]:
vw = vc.widget()
vw

VitessceWidget(config={'version': '1.0.15', 'name': 'Vitessce configuration for CellBrowser project adultPancr…